In [7]:
import pandas as pd
from datetime import datetime, timedelta

import yfinance as yf
import requests
import time

# Descargo Datos

In [8]:
# Yahoo Finance
yf_tickers = ['AAPL', 'GOOGL', 'MSFT', 'META']
start_date = '2025-01-01'
end_date = '2025-06-23'

def get_yf_prices(tickers, start, end):
    data = yf.download(tickers, start=start, end=end, group_by='ticker', auto_adjust=True, progress=False)
    daily_prices = []
    for ticker in tickers:
        df = data[ticker].reset_index()[['Date', 'Close']]
        df['asset_ticker'] = ticker
        df.rename(columns={'Date': 'date', 'Close': 'price_usd'}, inplace=True)
        daily_prices.append(df)
    return pd.concat(daily_prices, ignore_index=True)

yf_prices_df = get_yf_prices(yf_tickers, start_date, end_date)

In [9]:
# CoinGecko
def get_coingecko_prices(coin_id, days=365):
    url = f'https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart'
    params = {'vs_currency': 'usd', 'days': days, 'interval': 'daily'}
    r = requests.get(url, params=params)
    data = r.json()['prices']
    df = pd.DataFrame(data, columns=['timestamp', 'price_usd'])
    df['date'] = pd.to_datetime(df['timestamp'], unit='ms')
    df['asset_ticker'] = coin_id.upper()
    return df[['date', 'asset_ticker', 'price_usd']]

cg_cryptos = ['bitcoin', 'ethereum']
cg_dfs = []
for coin in cg_cryptos:
    cg_dfs.append(get_coingecko_prices(coin))
    time.sleep(1)

crypto_prices_df = pd.concat(cg_dfs)
crypto_prices_df['asset_ticker'] = crypto_prices_df['asset_ticker'].replace({'BITCOIN': 'BTC', 'ETHEREUM': 'ETH'})

In [13]:
import sqlite3
conn = sqlite3.connect('../local/examen/transactions.db')
transactions = pd.read_sql_query("SELECT * FROM main", conn)
print(transactions.shape)
transactions.head()

(125, 5)


,id,date,asset_ticker,type,quantity
0,1,2025-03-07,AAPL,BUY,275
1,2,2025-03-19,AAPL,BUY,284
2,3,2025-03-21,AAPL,BUY,256
3,4,2025-04-11,AAPL,SELL,381
4,5,2025-06-06,AAPL,BUY,129


# Ajusto Datos antes de enviar a BQ

In [16]:
yf_prices_df.head()

Price,date,price_usd,asset_ticker
0,2025-01-02,243.263199,AAPL
1,2025-01-03,242.774368,AAPL
2,2025-01-06,244.410416,AAPL
3,2025-01-07,241.627136,AAPL
4,2025-01-08,242.115936,AAPL


In [23]:
mask = crypto_prices_df['date'] <= yf_prices_df['date'].max()
mask.mean()
crypto_prices_df = crypto_prices_df[mask]
crypto_prices_df.head()

,date,asset_ticker,price_usd
0,2024-07-01,BTC,62734.393839
1,2024-07-02,BTC,62819.822269
2,2024-07-03,BTC,62042.565037
3,2024-07-04,BTC,60244.680764
4,2024-07-05,BTC,57189.174124


In [14]:
transactions = transactions[['date', 'asset_ticker', 'type', 'quantity']]
transactions.head()

,date,asset_ticker,type,quantity
0,2025-03-07,AAPL,BUY,275
1,2025-03-19,AAPL,BUY,284
2,2025-03-21,AAPL,BUY,256
3,2025-04-11,AAPL,SELL,381
4,2025-06-06,AAPL,BUY,129


# Envio a BQ

In [24]:
# Generamos conexion a BQ
from google.oauth2 import service_account # para generar conexion con BigQuery
import pandas_gbq # para interactuar con BQ
bq_cred = service_account.Credentials.from_service_account_file('../local/tokens/gcp_token_clase.json')

bq_cred

In [25]:
# Enviamos el df a BQ
pandas_gbq.to_gbq(yf_prices_df, 'ucema_2025.stocks', project_id= 'ucema-461612',
                   if_exists= 'replace', credentials = bq_cred)

pandas_gbq.to_gbq(crypto_prices_df, 'ucema_2025.crypto', project_id= 'ucema-461612',
                   if_exists= 'replace', credentials = bq_cred)

pandas_gbq.to_gbq(transactions, 'ucema_2025.transactions', project_id= 'ucema-461612',
                   if_exists= 'replace', credentials = bq_cred)